In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix



In [2]:
# Load the dataset
df = pd.read_csv('cleaned_data.csv')



In [3]:
# Define features and target
X = df.drop(['delay_rrt'], axis=1)
y = df['delay_rrt']




In [4]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



In [5]:
# Identify categorical columns
categorical_columns = X_train.select_dtypes(include=['object']).columns
numeric_columns = X_train.select_dtypes(include=['float64', 'int64']).columns



In [6]:
# Create preprocessor
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_columns),
    ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_columns)
])



In [7]:
# Create feature selector using XGBoost (as it gives feature importances)
feature_selector = SelectFromModel(
    XGBClassifier(n_estimators=100, random_state=42),
    max_features=20
)



In [8]:
# Create pipeline with SMOTE for handling class imbalance
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selector', feature_selector),
    ('smote', SMOTE(random_state=42)),
    ('classifier', XGBClassifier(random_state=42, scale_pos_weight=len(y_train[y_train == 0]) / len(y_train[y_train == 1])))
])



In [9]:
# Define a reduced hyperparameter space for RandomizedSearchCV
param_distributions = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [10, 20, None],
    'classifier__learning_rate': [0.01, 0.1, 0.2],
    'classifier__subsample': [0.8, 1.0],
    'classifier__colsample_bytree': [0.8, 1.0]
}



In [10]:
# Perform RandomizedSearchCV with cross-validation
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions,
    n_iter=10,  # Try 10 different combinations
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1,  # Use all available cores
    random_state=42
)



In [11]:
# Fit the model
random_search.fit(X_train, y_train)



RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               StandardScaler(),
                                                                               Index(['weight', 'aki_stage', 'gender', 'admission_age', 'hematocrit_min',
       'hematocrit_max', 'hemoglobin_min', 'hemoglobin_max', 'platelets_min',
       'platelets_max', 'wbc_min', 'wbc_max',...
                                                            multi_strategy=None,
                                                            n_estimators=None,
                                                            n_jobs=None,
                                                            num_parallel_tree=None,
                                                            random_state=42, ...))]),
                   n_jobs=-1,
                   param_distributions={'classifier__colsample_bytree': [0.8,
                                                                         1.0],
                                        'classifier__learning_rate': [0.01, 0.1,
                                                                      0.2],
                                        'classifier__max_depth': [10, 20, None],
                                        'classifier__n_estimators': [100, 200],
                                        'classifier__subsample': [0.8, 1.0]},
                   random_state=42, scoring='roc_auc')

In [12]:
# Get the best model
best_model = random_search.best_estimator_



In [13]:
# Function to evaluate model
def evaluate_model(model, X, y, dataset_name=""):
    y_pred = model.predict(X)
    y_pred_proba = model.predict_proba(X)[:, 1]
    
    print(f"\n{dataset_name} Results:")
    print("ROC AUC:", roc_auc_score(y, y_pred_proba))
    print("Classification Report:\n", classification_report(y, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y, y_pred))
    
    # Calculate precision-recall curve
    precision, recall, thresholds = precision_recall_curve(y, y_pred_proba)
    print("Average Precision Score:", average_precision_score(y, y_pred_proba))

# Evaluate on both train and test sets
evaluate_model(best_model, X_train, y_train, "Training")
evaluate_model(best_model, X_test, y_test, "Test")




Training Results:
ROC AUC: 0.9999803160584876
Classification Report:
               precision    recall  f1-score   support

           0       0.91      1.00      0.95       263
           1       1.00      0.98      0.99      1159

    accuracy                           0.98      1422
   macro avg       0.96      0.99      0.97      1422
weighted avg       0.98      0.98      0.98      1422

Confusion Matrix:
 [[ 263    0]
 [  25 1134]]
Average Precision Score: 0.9999955563290834

Test Results:
ROC AUC: 0.7228317659352143
Classification Report:
               precision    recall  f1-score   support

           0       0.39      0.53      0.45        66
           1       0.88      0.81      0.85       290

    accuracy                           0.76       356
   macro avg       0.64      0.67      0.65       356
weighted avg       0.79      0.76      0.77       356

Confusion Matrix:
 [[ 35  31]
 [ 54 236]]
Average Precision Score: 0.9047461983912692


In [14]:
# Get feature importance
def get_feature_importance(model, feature_names):
    # Get the feature importance from XGBoost
    if hasattr(model['classifier'], 'feature_importances_'):
        importance = model['classifier'].feature_importances_
        selected_features = model['feature_selector'].get_support()
        
        # Add selected features only
        feature_importance_dict = {}
        idx = 0
        for i, is_selected in enumerate(selected_features):
            if is_selected:
                feature_importance_dict[feature_names[i]] = importance[idx]
                idx += 1
        
        return sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Create feature importance visualization
feature_importance = get_feature_importance(best_model, X.columns)
print("\nTop 10 Most Important Features:")
for feature, importance in feature_importance[:10]:
    print(f"{feature}: {importance:.4f}")


Top 10 Most Important Features:
calcium_max: 0.0964
creatinine_min: 0.0789
aki_stage: 0.0736
aniongap_min: 0.0644
calcium_min: 0.0636
gender: 0.0614
pt_max: 0.0594
potassium_min: 0.0537
chloride_max: 0.0527
inr_max: 0.0524
